In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
%matplotlib inline
import cv2
import os
import math
from tqdm import tqdm
from mlxtend.image import extract_face_landmarks
# importing all the required libraries
import warnings
warnings.filterwarnings('ignore')
from skimage.transform import rotate, warp
from skimage.util import random_noise
from skimage.filters import gaussian
#Essential sklearn Functions
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
#Essential PyTorch libraries
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d 
from torch.nn import MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import SGD

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def eye_centers(landmarks):
    '''
    To find the eye centers(mean of the 6 landmark points around the eye)
    '''
    #36-41 are landmark points surrounding right eye
    point1 = (np.mean(landmarks[36:42,0]),np.mean(landmarks[36:42,1]))
    #42-47 are landmark points surrounding left eye
    point2 = (np.mean(landmarks[42:48,0]),np.mean(landmarks[42:48,1]))

    return point1, point2

In [ ]:
def find_angle(point1, point2):
    '''
    To find angle in degrees for the given two points
    '''
    # angle in radians
    angle_r = math.atan((point2[1] - point1[1])/(point2[0] - point1[0]))
    # angle in degrees
    angle_d = math.degrees(angle_r)

    return angle_d

In [ ]:
def rotate_image(image,angle):
    '''
    Returns a rotated image given the angle(degrees) to be rotated and image
    '''
    rows,cols = image.shape
    #Transformation Matrix(M)
    M = cv2.getRotationMatrix2D(((rows - 1)/2.0,(cols - 1)/2.0),angle,1)
    rot_img = cv2.warpAffine(image, M, (cols,rows))

    return rot_img

In [ ]:
def preprocessing(image_data):

    '''
    The preprocessing involves rotation of the image and image cropping
    Arguments :
    image_data -- array of images of shape (m,h,w)
    Returns :
    array of images after preprocessing
    '''
    preprocessed_faces = []
    for img in image_data:
        
        #landmark detection
        #(returns an array of landmarks of shape (68,2))
        landmarks = extract_face_landmarks(img)

        #detect eye cnters
        p1, p2 = eye_centers(landmarks)

        #find angle 
        angle = find_angle(p1, p2)
        
        #rotate image
        rot_img = rotate_image(img, angle)

        #find length 'd'
        p1_new, p2_new = eye_centers(extract_face_landmarks(rot_img))
        d = cv2.norm(np.array(p1_new) - np.array(p2_new))

        #mid point of new eye centers
        d_mid = ((p2_new[0]+p1_new[0])/2.0,(p2_new[1]+p1_new[1])/2.0)

        #point above line joining eye centers
        x_up = d_mid[0]
        y_up = d_mid[1] - (0.6*d)

        #cropping image
        x_start = int(landmarks[0,0])
        x_end = int(landmarks[16,0])
        y_start = int(y_up)
        y_end = int(landmarks[8,1])

        crop_img = img[y_start:y_end,x_start:x_end]

        #resize the cropped image
        face_roi = cv2.resize(crop_img,(48,48))
        
        preprocessed_faces.append(face_roi)

    return np.array(preprocessed_faces)

In [ ]:
def normalization(imagedata, mean, std_dev):
    '''
    To apply Histogram equalization and 
    Z-Square Normalization to the preprocessed images
    Arguments :
    imagedata -- array of preprocessed images of shape (m,h,w)
    mean -- mean of imagedata array
    std_dev -- standard deviation of imagedata array
    Returns :
    array of normalized images of shape (m,48,48)
    '''
    normalized_images = []
    for i in range(imagedata.shape[0]):
        #Histogram Equalization
        hist_eqv = cv2.equalizeHist(imagedata[i])

        #Z-Square normalization
        zsq_norm = ((hist_eqv - mean)/std_dev)

        #Resize
        resized_image = cv2.resize(zsq_norm, (48,48))
        normalized_images.append(resized_image)

    return np.array(normalized_images)

In [ ]:
def load_and_preprocess_data(dataset,num_classes,jaffe_dir_path = None,ck_dir_path = None):
    '''
    Loads the dataset given and also preprocesses it
    '''
    jaffe_data_list = []
    jaffe_labels_list = []
    ck_data_list = []
    ck_labels_list = []

    if dataset == 'jaffe' or dataset == 'combined':
        
        express_code = ['HA','AN','DI','FE','SA','SU','NE']
        for img in os.listdir(jaffe_dir_path):

            label = img[3:5]
            if num_classes == 6 and label == 'NE':
                continue                                              
            read_img = cv2.imread(jaffe_dir_path+img,
                                  cv2.IMREAD_GRAYSCALE)
            jaffe_data_list.append(read_img)
            jaffe_labels_list.append(express_code.index(label))

        jaffe_data = np.array(jaffe_data_list)
        jaffe_labels = np.array(jaffe_labels_list)
        jaffe_preprocessed_data = preprocessing(jaffe_data)
    if dataset == 'ck+' or dataset == 'combined':
        
        express_code = ['happy','anger','disgust','fear','sadness',
                        'surprise','contempt']
        for emcode in os.listdir(ck_dir_path):

            if num_classes == 6 and emcode == 'contempt':
                continue
            lst = os.listdir(ck_dir_path + emcode + '/')
            lst.sort()
            for i in range(2,len(lst),3):
                read_img = cv2.imread(ck_dir_path+emcode+'/'+lst[i],
                                      cv2.IMREAD_GRAYSCALE)
                ck_data_list.append(read_img)
                ck_labels_list.append(express_code.index(emcode))

        ck_data = np.array(ck_data_list)
        ck_labels = np.array(ck_labels_list)
        ck_preprocessed_data = preprocessing(ck_data)

    print('Data loading and Preprocessing is completed')

    if dataset == 'combined':
        X = np.concatenate((jaffe_preprocessed_data,ck_preprocessed_data),axis = 0)
        Y = np.concatenate((jaffe_labels,ck_labels),axis = 0)
        preprocessed_data, labels = shuffle(X,Y)
    elif dataset == 'ck+':
        preprocessed_data, labels = shuffle(ck_preprocessed_data, ck_labels)
    elif dataset == 'jaffe':
        preprocessed_data, labels = shuffle(jaffe_preprocessed_data, jaffe_labels)
    mean, std_dev = preprocessed_data.mean(),preprocessed_data.std()
    normalized_data = normalization(preprocessed_data, mean, std_dev)
    X_tmp = normalized_data.reshape(normalized_data.shape + (1,))
    #Y_tmp = to_categorical(labels)
    Y_tmp = labels
    print(X_tmp.shape)
    print(Y_tmp.shape)
    print('Normalization and Datareshaping is completed')

    return X_tmp,Y_tmp

In [ ]:
X_tmp, Y_tmp = load_and_preprocess_data('ck+',7,None,'drive/My Drive/Dataset Images/CK+48/')

Data loading and Preprocessing is completed
(327, 48, 48, 1)
(327,)
Normalization and Datareshaping is completed


In [ ]:
train_x, val_x, train_y, val_y = train_test_split(X_tmp, Y_tmp, test_size = 0.1, random_state = 6, shuffle = True)
print(train_x.shape, train_y.shape)
print(val_x.shape, val_y.shape)

(294, 48, 48, 1) (294,)
(33, 48, 48, 1) (33,)


In [ ]:
X_train = []
Y_train = []
for i in tqdm(range(train_x.shape[0])):
    X_train.append(train_x[i])
    X_train.append(rotate(train_x[i], angle=3))
    X_train.append(np.fliplr(train_x[i]))
    X_train.append(random_noise(train_x[i],var=0.2**2))
    for j in range(4):
        Y_train.append(train_y[i])

print('\n', len(Y_train), len(X_train))
X_train = np.array(X_train)
Y_train = np.array(Y_train)
print(X_train.shape, Y_train.shape)

100%|██████████| 294/294 [00:00<00:00, 2408.87it/s]


 1176 1176
(1176, 48, 48, 1) (1176,)


In [ ]:
# converting training & Validation images, labels into torch format
X_train = X_train.reshape(X_train.shape[0], 1, 48, 48)
X_train  = torch.from_numpy(X_train)
X_train = X_train.float()

Y_train = Y_train.astype(int)
Y_train = torch.from_numpy(Y_train)

val_x = val_x.reshape(val_x.shape[0], 1, 48, 48)
val_x  = torch.from_numpy(val_x)
val_x = val_x.float()

val_y = val_y.astype(int)
val_y = torch.from_numpy(val_y)

In [ ]:
torch.manual_seed(42)

class Net(Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = Sequential(
            
            # Defining 1st 2D convolution layer
            Conv2d(1, 48, kernel_size=5, stride=1, padding=0),
            ReLU(inplace=True),
            BatchNorm2d(48),
            MaxPool2d(kernel_size=2, stride=2, padding=0),

            # Defining 2nd 2D convolution layer
            Conv2d(48, 64, kernel_size=5, stride=1, padding=0),
            ReLU(inplace=True),
            BatchNorm2d(64),
            MaxPool2d(kernel_size=2, stride=2, padding=0),
        )

        self.linear_layers = Sequential(
            Dropout(p=0.5),
            Linear(64 * 9 * 9, 7),
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(-1, self.num_flat_features(x))
        x = self.linear_layers(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [ ]:
model = Net()
optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = CrossEntropyLoss()

# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    
print(model)

Net(
  (cnn_layers): Sequential(
    (0): Conv2d(1, 48, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(48, 64, kernel_size=(5, 5), stride=(1, 1))
    (5): ReLU(inplace=True)
    (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layers): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=5184, out_features=7, bias=True)
  )
)


In [ ]:
torch.manual_seed(42)

# batch size
batch_size = 16

# (#epochs)
n_epochs = 100

for epoch in range(1, n_epochs+1):

    train_loss = 0.0

    #random permutation of training indices
    permutation = torch.randperm(X_train.size()[0])

    training_loss = []
    for i in tqdm(range(0,X_train.size()[0], batch_size)):
        #forming batches
        indices = permutation[i:i+batch_size]
        batch_x, batch_y = X_train[indices], Y_train[indices]
        
        if torch.cuda.is_available():
            batch_x, batch_y = batch_x.cuda(), batch_y.cuda()
        
        optimizer.zero_grad() #making the parameters zero before training
        outputs = model(batch_x)
        loss = criterion(outputs,batch_y) #criterion used is CrossEntropy

        training_loss.append(loss.item())
        loss.backward() #backpropagation
        optimizer.step() #Does the update
        
    training_loss = np.average(training_loss)
    print('epoch: \t', epoch, '\t training loss: \t', training_loss)

 41%|████      | 30/74 [00:00<00:00, 298.28it/s]

epoch: 	 1 	 training loss: 	 0.759352172448023


 53%|█████▎    | 39/74 [00:00<00:00, 388.16it/s]

epoch: 	 2 	 training loss: 	 0.2876296406989363


100%|██████████| 74/74 [00:00<00:00, 390.95it/s]


...

epoch: 	 97 	 training loss: 	 0.00040679827766571425
epoch: 	 98 	 training loss: 	 0.00019536184404571672



 55%|█████▌    | 41/74 [00:00<00:00, 400.67it/s]

epoch: 	 99 	 training loss: 	 0.000443612160361718


100%|██████████| 74/74 [00:00<00:00, 388.39it/s]

epoch: 	 100 	 training loss: 	 0.0003366751499734656


In [ ]:
torch.save(model, 'model.pt')

In [ ]:
the_model = torch.load('model.pt')

In [ ]:
torch.manual_seed(42)

# prediction for training set
prediction = []
target = []

permutation = torch.randperm(X_train.size()[0])

for i in tqdm(range(0,X_train.size()[0], batch_size)):
    
    indices = permutation[i:i+batch_size]
    batch_x, batch_y = X_train[indices], Y_train[indices]

    if torch.cuda.is_available():
        batch_x, batch_y = batch_x.cuda(), batch_y.cuda()

    with torch.no_grad():
        output = model(batch_x.cuda())

    softmax = torch.exp(output).cpu()
    prob = list(softmax.numpy()) #list of probabilities
    predictions = np.argmax(prob, axis=1) #label with highest prob
    prediction.append(predictions)
    target.append(batch_y)
    
# training accuracy
accuracy = []
for i in range(len(prediction)):
    accuracy.append(accuracy_score(target[i].cpu(),prediction[i]))
    
print('training accuracy: \t', np.average(accuracy))

100%|██████████| 74/74 [00:00<00:00, 789.61it/s]

training accuracy: 	 1.0


In [ ]:
#val accuracy
torch.manual_seed(42)
output = model(val_x.cuda())

softmax = torch.exp(output).cpu()
prob = list(softmax.detach().numpy())
predictions = np.argmax(prob, axis=1)
accuracy_score(val_y, predictions)

0.9090909090909091